In [ ]:
suppressMessages({
    library(tidyverse)
    library(data.table)
    library(ggpubr)
    library(RColorBrewer)

})


save_plot <- function(p, fn, w, h){
    for(ext in c(".pdf", ".png")){
        ggsave(filename=paste0(fn,ext), plot=p, width=w, height=h)
    }
}

dir.create('../_m/')
setwd('../_m/')

In [ ]:
# create a list of all the filenames in the folder that end with ".csv"
file_list <- Sys.glob('../../_m/*_gene_set_results.gsa.out')
study_name <- gsub('.*_m/|_gene_set.*','',file_list)

df <- data.frame()

for (i in seq_along(file_list)){
    dftmp <- fread(file_list[i], skip=4) %>%
                    set_names(c('VARIABLE','TYPE','NGENES','BETA','BETA_STD','SE','P')) %>%
                    mutate(GWAS_set = study_name[i],
                           `-log10P` = 0-log10(P),
                           fdr_bh = p.adjust(P, method = "BH"),
                           fdr_bonferroni = p.adjust(P, method = "bonferroni"),
                           Status = ifelse(grepl('down',VARIABLE), 'Downregulated','Upregulated'),
                           `Brain Region` = ifelse(grepl('caudate',VARIABLE),'Caudate',
                                                  ifelse(grepl('dlpfc', VARIABLE),'DLPFC',
                                                      'Hippocampus'))
                          ) %>%
                    mutate(GWAS_set = ifelse(GWAS_set == 'SCZ', 'SCHIZOPHRENIA', GWAS_set)) %>%
                    select(-TYPE)
                           
                       
                       
    df <- rbind(dftmp,df)
                    
    }


df %>% head()

fwrite(df, 'magma_enrichment_results.tsv',quote=F,sep='\t',row.names=F)



In [ ]:
head(df)

In [ ]:
df %>% filter(P < 0.05)

df %>% filter(fdr_bh < 0.05)

In [ ]:
# Define a custom color palette using RColorBrewer
my_palette <- rev(brewer.pal(9, "YlGnBu"))

# Define a custom color scale using scale_fill_gradientn
my_scale <- scale_fill_gradientn(colors = my_palette,
                                 values = c(-1.8, 0, 1.8),
                                 name = "BETA",
                                 guide = guide_colorbar(barwidth = 10, barheight = 0.5))

In [ ]:
# ggplot(df, aes(x = VARIABLE, y = GWAS_set, size = BETA, fill = `-log10P`)) +
#     #scale_fill_gradient(low = "blue", high = "red") +
#     geom_point(shape = 21) +
#     xlab("Gene Set") +
#     ylab("") +
#     theme_pubr() +
#     theme(axis.text.x = element_text(angle = 45, hjust = 1)) 

In [ ]:
# ggplot(df, aes(x = VARIABLE, y = GWAS_set, fill = BETA, size = `-log10P`)) +
#     #scale_fill_gradient(low = "blue", high = "red") +
#     geom_point(shape = 21) +
#     xlab("Gene Set") +
#     ylab("") +
#     theme_pubr()

In [ ]:
# ggplot(data = df,
#        aes(y = BETA,
#            x = `-log10P`)) +
#   geom_point(size = 1.5) +
#   geom_hline(yintercept = 0,
#              colour = "royalblue") +
#   geom_vline(xintercept = 0,
#              colour = "royalblue") +
#   xlim(0, 7) +
#   theme_bw() +
#   facet_grid(~VARIABLE)

In [ ]:
options(repr.plot.width = 13, repr.plot.height = 10, repr.plot.res = 100)


p1 <- ggplot(data = df, aes(y = BETA, x = `-log10P`)) + 
            geom_point(aes(color = Status, size = NGENES)) +
            geom_hline(yintercept = 0, colour = "royalblue", linewidth=1) +
            geom_vline(xintercept = 1.3010, colour = "goldenrod2",
                       linewidth=1,linetype = "dashed") +
            ylim(-1, 1) +
            theme_bw() +
            facet_grid(`Brain Region`~GWAS_set) +
            scale_color_manual(values = c("green3", "firebrick")) +
            scale_size(range = c(1.5, 4), 
                       labels = c('3', '60', '120','340'), 
                       guide = guide_legend(title = "Number of Genes", 
                                            label.position = "right")) +
            theme(panel.grid.major.x = element_blank(), 
                  panel.grid.minor.x = element_blank(),
                  panel.grid.major.y = element_blank(),
                  panel.grid.minor.y = element_blank())

p1

save_plot(p1, 'magma_enrichment', 12, 8)



In [ ]:
sessionInfo()